## Initial Setup


In [1]:
from dotenv import load_dotenv

load_dotenv()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

## Creating a weather forecasting agent

In the next cells, we build the logic for a simple agent that can answer questions about the weather. The code for the agent is adapted from Peter Roelants's excellent blog post ["Implement a simple ReAct Agent using OpenAI function calling"](https://peterroelants.github.io/posts/react-openai-function-calling/).

First, we create a system prompt that initiates some basic ReAct agent logic:


In [9]:
system_prompt = """
You are a helpful assistant who can answer multistep questions by sequentially calling functions. 

Follow a pattern of:
THOUGHT (reason step-by-step about which function to call next),
ACTION (call a function to as a next step towards the final answer), 
OBSERVATION (output of the function).

Reason step by step which actions to take to get to the answer. 
Only call functions with arguments coming verbatim from the user or the output of other functions.
"""


def get_initial_messages(question_prompt):
    return [
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": question_prompt,
        },
    ]

Next, we define some tools that we want the agent to have access to:


In [10]:
import json
import requests
import time

FORECAST_API_URL = "https://api.open-meteo.com/v1/forecast"
CURRENT_LOCATION_BY_IP_URL = "http://ip-api.com/json?fields=lat,lon"


def get_current_location():
    time.sleep(0.5)  # simulate a longer task
    print(requests.get(CURRENT_LOCATION_BY_IP_URL).json())
    return json.dumps(requests.get(CURRENT_LOCATION_BY_IP_URL).json())


def get_current_weather(latitude, longitude, temperature_unit):
    time.sleep(0.3)  # simulate a longer task
    resp = requests.get(
        FORECAST_API_URL,
        params={
            "latitude": latitude,
            "longitude": longitude,
            "temperature_unit": temperature_unit,
            "current_weather": True,
        },
    )
    return json.dumps(resp.json())


def calculate(formula):
    return str(eval(formula))


class StopException(Exception):
    """
    Signal that the task is finished.
    """


def finish(answer):
    raise StopException(answer)


available_functions = {
    "get_current_location": get_current_location,
    "get_current_weather": get_current_weather,
    "calculate": calculate,
    "finish": finish,
}

Next, we define a JSON schema in the `function_schema` array to describe each function. We'll pass this to the agent:


In [11]:
function_schema = [
    {
        "function": {
            "name": "get_current_location",
            "description": "Get the current location of the user.",
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
        "type": "function",
    },
    {
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                    "temperature_unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["latitude", "longitude", "temperature_unit"],
            },
        },
        "type": "function",
    },
    {
        "function": {
            "name": "calculate",
            "description": "Calculate the result of a given formula.",
            "parameters": {
                "type": "object",
                "properties": {
                    "formula": {
                        "type": "string",
                        "description": "Numerical expression to compute the result of, in Python syntax.",
                    }
                },
                "required": ["formula"],
            },
        },
        "type": "function",
    },
    {
        "function": {
            "name": "finish",
            "description": "Once you have the information required, answer the user's original question, and finish the conversation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "answer": {
                        "type": "string",
                        "description": "Answer to the user's question.",
                    }
                },
                "required": ["answer"],
            },
        },
        "type": "function",
    },
]

We instantiate our OpenAI client:


In [12]:
from openai import OpenAI
import os

oai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

We create a function called `execute_loop_step` that handles the recursive agent logic. In the function, we:

1. Call the OpenAI chat completions endpoint with the system and user prompt.
2. Execute any tool calls requested by the LLM.
3. Add the results of those tool calls to the messages array in the `append_tool_message_and_execute_loop` helper function.
4. Call the chat completions endpoint with the new messages array.
5. We stop the loop when we either have an answer, or we've reached the `MAX_CALLS` limit.


In [13]:
from ddtrace.llmobs.decorators import *

MAX_CALLS = 4
MODEL = "gpt-4"


@workflow()
def execute_loop_step(messages, calls_left=MAX_CALLS):

    if calls_left < 1:
        return messages

    # https://platform.openai.com/docs/api-reference/chat/create#chat-create-messages
    response = oai_client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=function_schema,
    )
    response_message = response.choices[0].message
    if response_message.content:
        print("\n")
        print(response_message.content)
    if response_message.tool_calls:
        print("\n")
        print("CALL TOOL:", response_message.tool_calls)
    messages.append(response_message)
    if not response_message.tool_calls:
        return execute_loop_step(messages, calls_left - 1)

    for tool_call in response_message.tool_calls:
        # define a small helper function to reduce repetitive code
        def append_tool_message_and_execute_loop(content):
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "content": content,
                }
            )
            return execute_loop_step(messages, calls_left - 1)

        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        if function_to_call is None:
            return append_tool_message_and_execute_loop(
                f"Invalid function name: {function_name!r}"
            )
        try:
            function_args_dict = json.loads(tool_call.function.arguments)
        except json.JSONDecodeError as exc:
            return append_tool_message_and_execute_loop(
                f"Error decoding function call `{function_name}` arguments {tool_call.function.arguments!r}! Error: {exc!s}"
            )
        try:
            with LLMObs.tool(function_name):
                LLMObs.annotate(input_data=function_args_dict)
                try:
                    function_response = function_to_call(**function_args_dict)
                    LLMObs.annotate(output_data=function_response)
                except StopException as answer:
                    LLMObs.annotate(output_data="StopException")
                    return str(answer)
            return append_tool_message_and_execute_loop(function_response)
        except Exception as exc:
            return append_tool_message_and_execute_loop(
                f"Error calling function `{function_name}`: {type(exc).__name__}: {exc!s}!"
            )
    return "no answer found"

Finally, we create the top-level function to take a prompt from a user, call the agent, and return a response:


In [14]:
# https://docs.datadoghq.com/tracing/llm_observability/sdk/#agent-span
@agent()
def call_weather_assistant(question_prompt):
    LLMObs.annotate(
        input_data=question_prompt,
    )
    messages = get_initial_messages(question_prompt)
    answer = execute_loop_step(messages)
    LLMObs.annotate(
        output_data=answer,
    )
    return answer

Now, we can ask the weather assistant questions:


In [15]:
call_weather_assistant(
    "What is the weather in my current location? Please give me the temperature in farenheit. Also tell me my current location coordinates."
)



CALL TOOL: [ChatCompletionMessageToolCall(id='call_HfZSK8FEQwmVVYRNqLyh9FfH', function=Function(arguments='{}', name='get_current_location'), type='function')]
{'lat': 41.7599, 'lon': -72.7574}


THOUGHT
Now that I have obtained the user's current location coordinates, I will use these coordinates to get the weather data at that location. The user wants the temperature in Fahrenheit, so I will specify that in the function.

ACTION
Call the function to get the current weather of the user's location.


CALL TOOL: [ChatCompletionMessageToolCall(id='call_md31coobmrNryqsSj0AqgO7s', function=Function(arguments='{\n"latitude": 41.7599,\n"longitude": -72.7574,\n"temperature_unit": "fahrenheit"\n}', name='get_current_weather'), type='function')]


THOUGHT
Now that I have the weather data, I will respond to the user with the temperature and their location coordinates. 

ACTION
Finish the request and provide the requested data to the user.


CALL TOOL: [ChatCompletionMessageToolCall(id='call_Nq

'The current temperature in your location (41.7599, -72.7574) is 95.6°F.'

## Viewing the trace in Datadog

Now, try checking out the [LLM Observability interface](https://app.datadoghq.com/llm) in Datadog. You should see a trace that describes the agent we just ran.
